In [61]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad", use_fast=False)
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad", return_dict=False)

bi_encoder = SentenceTransformer('nq-distilbert-base-v1')

from inspect import getsourcefile
import os.path as path, sys
current_dir = path.dirname(path.abspath(getsourcefile(lambda:0)))
sys.path.insert(0, current_dir[:current_dir.rfind(path.sep)])
import src.clean_dataset as clean

import pandas as pd
from sentence_transformers import SentenceTransformer, util

import time

In [82]:
#import data:
df = pd.read_csv('../data/processed/taxonomy_final.csv')

1344


In [63]:
splitted = clean.split_at_length(df, 'all_text', 512)
passages = splitted.text.tolist()
corpus_embeddings = bi_encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/445 [00:00<?, ?it/s]

In [102]:
import pickle5 as pickle
pickle.dump(corpus_embeddings, open("../data/processed/splitted_corpus_embeddings.pkl", "wb" ))

In [87]:
splitted = clean.split_at_length(df, 'all_text', 1000)
passages_long = splitted.text.tolist()
print(len(passages_long))
corpus_embeddings_long = bi_encoder.encode(passages_long, convert_to_tensor=True, show_progress_bar=True)

11621


Batches:   0%|          | 0/364 [00:00<?, ?it/s]

In [99]:
query = input()

top_k = 5  # Number of passages we want to retrieve with the bi-encoder
start_time = time.time()
question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
hits = hits[0]  # Get the hits for the first query
end_time = time.time()

# Output of top-k hits
print("Input question:", query)
print("Results (after {:.3f} seconds):".format(end_time - start_time))
matches = []
for hit in hits:
    print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']]))
    matches.append(passages[hit['corpus_id']])
print("\n\n========\n")

what is done for LED lightning in Viet nam?
Input question: what is done for LED lightning in Viet nam?
Results (after 0.250 seconds):
	0.515	The project will expand the existing PA system through the establishment of one new terrestrial PA and two new marine PAs (the first marine PAs in the country).  In addition
	0.480	the project will address systemic issues by implementing key elements of SKNs Protected Areas System Plan
	0.477	and EE. It will also include RE mini-grid how-to handbooks tailored to PNG
	0.473	including the establishment of a centralized agency for protected areas management
	0.406	ficing for renewable energy and energy efficiency initiatives in the energy





In [97]:
matches[0]

'The objective of the Project is to mitigate GHG emissions through transformation of the lighting market towards greater usage of locally produced LED lighting products in Viet Nam. This objective will be achieved by removing barriers to increased production and utilization of locally produced LED lighting products in Vietnam through two components: i) the transfer of skills'

In [98]:
inputs = tokenizer.encode_plus(query, matches[0], add_special_tokens=True, return_tensors="pt")
input_ids = inputs["input_ids"].tolist()[0]

text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
answer_start_scores, answer_end_scores = model(**inputs)

answer_start = torch.argmax(answer_start_scores)  # Get the most likely beginning of answer with the argmax of the score
answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score

answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

print(f"Question: {query}")
print(f"Answer: {answer}\n")

Question: what is done for LED  lightning in Vietnam?
Answer: removing barriers to increased production and utilization of locally produced led lighting products

